In [294]:
%matplotlib widget

In [295]:
import numpy as np
import matplotlib.pyplot as plt
import pydae.ssa as ssa
import scipy.signal as sctrl

In [296]:
from vsc_lcl import vsc_lcl_class 

## Instantiate system

In [297]:
syst = vsc_lcl_class()
syst.Dt = 5e-6
syst.decimation = 1
syst.N_store = 100_000
syst.update()

## CTRL1 in state feedback

In [298]:
syst = vsc_lcl_class()
syst.Dt = 5e-6
syst.decimation =1
syst.N_store =100_000
syst.update()

Δt = 50e-6 
#x_d_ctrl_list = ['i']   # states to consider in the reduction
z_ctrl_list = [ 'i_sd_g01', 'i_sq_g01']     # outputs to consider in the controller
u_ctrl_list = ['eta_d_g01','eta_q_g01']     # intputs to consider in the controller
z_ctrl_idxs = [syst.outputs_list.index(item) for item in z_ctrl_list]
u_ctrl_idxs = [syst.inputs_run_list.index(item) for item in u_ctrl_list]
u_pert_list = ['v_sD_g01','v_sQ_g01']     # intputs to consider in the controller
u_pert_idxs = [syst.outputs_list.index(item) for item in z_pert_list]

syst.Δt = Δt

## Calculate equilibirum point
syst.initialize([{'G_d_g01':0.0,'eta_d_g01':0.0,'eta_q_g01':-0.8693333,'v_1_Q':-326,'v_1_D':0.0, 'C_m_g01':4e-6}],xy0=1000)
ssa.eval_ss(syst)

# linear continous plant
A_p = syst.A
B_p = syst.B
C_p = syst.C
D_p = syst.D

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_p,B_p,C_p,D_p),Δt,method='zoh')

N_z_d,N_x_d = C_d.shape  # discreticed plant dimensions
N_x_d,N_u_d = B_d.shape

# convenient matrices
O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)

syst.A_d = A_d
syst.B_d = B_d


# Controller ##################################################################################
B_c = B_d[:,u_ctrl_idxs]
C_c = C_d[z_ctrl_idxs,:]
D_c = D_d[z_ctrl_idxs,:][:,u_ctrl_idxs]

N_x_c,N_u_d = B_c.shape
N_z_c,N_x_c = C_c.shape


O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref
omega_b = 2*np.pi*50

W =  np.block([
                [   np.cos(omega_b*Δt), -np.sin(omega_b*Δt)],   
                [   np.sin(omega_b*Δt),  np.cos(omega_b*Δt)],    
               ])
W = np.eye(2)
A_e = np.block([
                [    A_d,  B_c@W, O_xu],    # Δx_d
                [   O_ux,   O_uu, O_uu],    # Δx_r
                [ Δt*C_c, Δt*D_c, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])

A_ctrl = A_e[N_x_d:,N_x_d:]
B_ctrl = B_e[N_x_d:]

# weighting matrices
Q_c = np.eye(A_e.shape[0])
Q_c[-1,-1] = 1e6
Q_c[-2,-2] = 1e6

R_c = np.eye(B_c.shape[1])*100000

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)

E_cont = np.log(E_c)/Δt

syst.A_ctrl = A_ctrl
syst.B_ctrl = B_ctrl
syst.K_c = K_c
syst.N_x_d = N_x_d  # number of plant states
syst.N_u_d = N_u_d  # number of plant inputs
syst.N_z_c = N_z_c  # number of plant outputs considered for the controller

## Observer

In [349]:
z_obs_list =  ['v_md_g01', 'v_mq_g01','i_sd_g01', 'i_sq_g01']       # outputs to consider in the observer
z_obs_idxs  = [syst.outputs_list.index(item) for item in z_obs_list]

# discretized plant:
# Dx_d = A_d*Dx_d + B_d*Du_d
# z_o  = C_o*Dx_d + D_o*Du_d

# x_o = A_d*x_o + B_d*u_d + L_o*(z_o - C_o*x_o - D_o*Du_d)
B_o = B_d[:,u_ctrl_idxs]
C_o = C_d[z_obs_idxs,:]
D_o = D_d[z_obs_idxs,:][:,u_ctrl_idxs]
B_pert = B_d[:,[1,2]]
N_z_o = C_o.shape[0]

Q_o = np.eye(A_d.shape[0])

R_o = np.diag([1]*N_z_o)
K_o_T,S_o,E_o = ssa.dlqr(A_d.T,C_o.T,Q_o,R_o)
K_o = K_o_T.T

syst.K_o = K_o
syst.C_o = C_o
syst.D_o = D_o

syst.z_obs_list = z_obs_list
syst.N_z_o = N_z_o
syst.x_obs = np.zeros((N_x_d,1))

print('damp_ctrl',-E_c.real/np.abs(E_c))
print('damp_obs',-E_o.real/np.abs(E_o))

damp_ctrl [-0.48951411 -0.48951411 -0.51666281 -0.51666281 -0.99964555 -0.99964555
 -0.9999404  -0.9999404  -1.          1.        ]
damp_obs [-0.99987663 -0.99987663 -0.11391326 -0.11391326 -0.08265076 -0.08265076]


## Performance assessment

In [350]:
syst.Dt = 5e-6
syst.decimation =1
syst.N_store =100_000
syst.update()
times = np.arange(0.0,0.1,Δt)

syst.initialize([{'G_d_g01':0.0,'eta_d_g01':0.0,'eta_q_g01':-0.8693333,'v_1_Q':-326,'v_1_D':0.0, 'C_m_g01':4e-6}],xy0=1000)
ssa.eval_A(syst)
i_sd = syst.get_value('i_sd_g01')
i_sq = syst.get_value('i_sq_g01')  
v_sd = syst.get_value('v_sd_g01')
v_sq = syst.get_value('v_sq_g01')
i_td = syst.get_value('i_td_g01')
i_tq = syst.get_value('i_tq_g01') 
v_md = syst.get_value('v_md_g01')
v_mq = syst.get_value('v_mq_g01') 
v_dc = syst.get_value('v_dc_g01')
eta_d = syst.get_value('eta_d_g01')
eta_q = syst.get_value('eta_q_g01')
i_sd_ref_0 = i_sd
i_sq_ref_0 = i_sq
v_sq_0 = v_sq
v_sd_0 = v_sd
x_d_0 = np.array([i_td,i_tq,v_md,v_mq,i_sd,i_sq]).reshape(6,1)
u_d_0 = np.array([eta_d,eta_q]).reshape(2,1)
x_r_0 = u_d_0
syst.Δx_e = np.zeros((10,1))

# outputs for observer         
z_obs_values_list = []
for item in syst.z_obs_list:
     z_obs_values_list += [syst.get_value(item)]
z_o_0 = np.array(z_obs_values_list).reshape(N_z_o,1)
syst.Δx_o = np.zeros((6,1))    
ΔX_o = np.zeros((len(times),6))

it = 0
for t in times:
    
    Δx_e = syst.Δx_e
    Δx_o = syst.Δx_o
    
    # measurements
    i_sd = syst.get_value('i_sd_g01')
    i_sq = syst.get_value('i_sq_g01')  
    v_sd = syst.get_value('v_sd_g01')
    v_sq = syst.get_value('v_sq_g01')
    i_td = syst.get_value('i_td_g01')
    i_tq = syst.get_value('i_tq_g01') 
    v_md = syst.get_value('v_md_g01')
    v_mq = syst.get_value('v_mq_g01') 
    v_dc = syst.get_value('v_dc_g01')

    x_d = np.array([i_td,i_tq,v_md,v_mq,i_sd,i_sq]).reshape(6,1)
    
    Δx_d = x_d - x_d_0 
    Δx_d = Δx_o
    Δx_r = syst.Δx_e[N_x_c:-N_u_d,:]   
    Δx_i = syst.Δx_e[(N_x_c+N_u_d):,:] 

    # outputs for observer         
    z_obs_values_list = []
    for item in syst.z_obs_list:
         z_obs_values_list += [syst.get_value(item)]
    z_o = np.array(z_obs_values_list).reshape(N_z_o,1)
    
    Δz_o = z_o - z_o_0
    
    # observer dynamics
    Δu_pert = np.block([[ (v_sd-v_sd_0)],[(v_sq-v_sq_0)]])
    
    Δx_o = A_d @ Δx_o + B_o@(Δx_r) + B_pert@Δu_pert  +  K_o @ (Δz_o - C_o @ Δx_o - D_o @ (Δx_r-Δx_pert))
    
    
    i_sd_ref = i_sd_ref_0
    i_sq_ref = i_sq_ref_0
    v_sq = v_sq_0
    v_sd = v_sd_0
    if t>20e-3: i_sd_ref = 20
    if t>30e-3: i_sq_ref = 30
    if t>45e-3: v_sd =  163 
    if t>45e-3: v_sq = -163
    epsilon_d = i_sd - i_sd_ref
    epsilon_q = i_sq - i_sq_ref  
    
    epsilon = np.block([[epsilon_d],[epsilon_q]])
    
    Δu_r = -K_c @ Δx_e + np.block([[ (v_sd-v_sd_0)*2/v_dc],[(v_sq-v_sq_0)*2/v_dc]])
    
   
    Δx_r = Δu_r
    Δx_i += Δt*epsilon
    
    Δx_e = np.block([[Δx_d],[Δx_r],[Δx_i]])
    
    
    
    x_r = Δx_r + x_r_0 
    
    eta_dq = x_r
    eta_d = eta_dq[0,0]  
    eta_q = eta_dq[1,0]   

    
    events=[{'t_end':t,'eta_d_g01':eta_d,'eta_q_g01':eta_q,'v_1_Q':v_sq,'v_1_D':v_sd}]
    syst.run(events)

#    eta_d_prev = eta_d
#    eta_q_prev = eta_q

    syst.Δx_e =  Δx_e
    syst.Δx_o =  Δx_o
    
    ΔX_o[it,:] = Δx_o.T
    
    it += 1
    
syst.post();



In [351]:
plt.close('all')
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7),sharex=True)

lines = axes[0].plot(syst.T,syst.get_values('i_sd_g01'),label='i_sd_g01')
lines = axes[0].plot(syst.T,syst.get_values('i_sq_g01'),label='i_sq_g01')

axes[1].plot(syst.T,syst.get_values('eta_D_g01'),label='eta_D_g01')
axes[1].plot(syst.T,syst.get_values('eta_Q_g01'),label='eta_Q_g01')
for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

In [352]:
plt.close('all')
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(7, 7),sharex=True)

axes[0].plot(syst.T,syst.get_values('i_td_g01'),label='i_td')
axes[0].plot(syst.T,syst.get_values('i_tq_g01'),label='i_tq')
axes[0].plot(times,ΔX_o[:,0],label='i_sd est.')
axes[0].plot(times,ΔX_o[:,1],label='i_sq est.')

axes[1].plot(syst.T,syst.get_values('v_md_g01'),label='v_md')
axes[1].plot(syst.T,syst.get_values('v_mq_g01'),label='v_mq')
axes[1].plot(times,ΔX_o[:,2]+syst.get_values('v_md_g01')[0],label='v_md est.')
axes[1].plot(times,ΔX_o[:,3]+syst.get_values('v_mq_g01')[0],label='v_mq est.')

axes[2].plot(syst.T,syst.get_values('i_sd_g01'),label='i_sd')
axes[2].plot(syst.T,syst.get_values('i_sq_g01'),label='i_sq')
axes[2].plot(times,ΔX_o[:,4]+syst.get_values('i_sd_g01')[0],label='i_sd est.')
axes[2].plot(times,ΔX_o[:,5]+syst.get_values('i_sq_g01')[0],label='i_sq est.')


for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

## Simulink

In [353]:
import sympy as sym

x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6 = sym.symbols('Dx_d_1,Dx_d_2,Dx_d_3,Dx_d_4,Dx_d_5,Dx_d_6')
x_r_1,x_r_2 = sym.symbols('Dx_r_1,Dx_r_2')
x_i_1,x_i_2 = sym.symbols('Dx_i_1,Dx_i_2')

x_e = sym.Matrix([x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6,x_r_1,x_r_2,x_i_1,x_i_2])
u_r = -K_c * x_e

u_r_d = str(sym.N(u_r[0],8))
u_r_q = str(sym.N(u_r[1],8))

print(f'Du_r_1 = {u_r_d};')
print(f'Du_r_2 = {u_r_q};')
print('\nWarning: Control output is eta_dq!!')

Du_r_1 = -0.013361535*Dx_d_1 - 0.00027748637*Dx_d_2 + 0.0021866204*Dx_d_3 + 5.2865939e-5*Dx_d_4 + 0.0071983421*Dx_d_5 + 0.00022706741*Dx_d_6 - 2.2443754*Dx_i_1 + 0.68967688*Dx_i_2 - 0.36460717*Dx_r_1 - 0.0049366387*Dx_r_2;
Du_r_2 = 0.00027748637*Dx_d_1 - 0.013361535*Dx_d_2 - 5.2865939e-5*Dx_d_3 + 0.0021866204*Dx_d_4 - 0.00022706741*Dx_d_5 + 0.0071983421*Dx_d_6 - 0.68967688*Dx_i_1 - 2.2443754*Dx_i_2 + 0.0049366387*Dx_r_1 - 0.36460717*Dx_r_2;



In [359]:
it_ini = 4
Δx_o = sym.Matrix([sym.Symbol(f'xD[{it+it_ini}]') for it in range(6)])
Δz_o = sym.Matrix([sym.Symbol(item) - sym.Symbol(item+'_0') for item in ['v_md', 'v_mq', 'i_sd', 'i_sq']])
Δx_r = sym.Matrix([sym.Symbol(f'Dx_r_{it+1}') for it in range(2)])
Δu_pert = sym.Matrix([sym.Symbol(f'Du_pert_{it+1}') for it in range(2)])

Δx_o_kp1 = A_d @ Δx_o + B_o@(Δx_r) +  K_o @ (Δz_o - C_o @ Δx_o) # + B_pert@Δu_pert  +  K_o @ (Δz_o - C_o @ Δx_o - D_o @ (Δx_r))

for it in range(6):
    print(f'xD[{it+it_ini}] = {Δx_o_kp1[it]};')

xD[4] = 13.7996619184135*Dx_r_1 + 0.103767816974477*Dx_r_2 + 0.290189894236834*i_sd - 0.290189894236834*i_sd_0 + 0.00455866714091358*i_sq - 0.00455866714091358*i_sq_0 + 0.004003346842446*v_md - 0.004003346842446*v_md_0 + 6.2889597697231e-5*v_mq - 6.2889597697231e-5*v_mq_0 + 0.768729374614947*xD[4] + 0.0120761660206275*xD[5] - 0.0376316155873075*xD[6] - 0.000591164657503082*xD[7] - 0.060612006008949*xD[8] - 0.000952169478073937*xD[9];
xD[5] = -0.103767816974477*Dx_r_1 + 13.7996619184135*Dx_r_2 - 0.00455866714091368*i_sd + 0.00455866714091368*i_sd_0 + 0.290189894236833*i_sq - 0.290189894236833*i_sq_0 - 6.28895976972104e-5*v_md + 6.28895976972104e-5*v_md_0 + 0.00400334684244601*v_mq - 0.00400334684244601*v_mq_0 - 0.0120761660206275*xD[4] + 0.768729374614947*xD[5] + 0.000591164657503061*xD[6] - 0.0376316155873075*xD[7] + 0.000952169478074047*xD[8] - 0.060612006008949*xD[9];
xD[6] = 86.1438235366479*Dx_r_1 + 0.886477352734974*Dx_r_2 - 4.2224902133574*i_sd + 4.2224902133574*i_sd_0 - 0.066332

In [354]:
Du_r_1,Du_r_2 = sym.symbols('Du_r_1,Du_r_2')
Du_r = sym.Matrix([Du_r_1,Du_r_2 ])

Dx_r = W@Du_r

Dx_r_1 = str(sym.N(Dx_r[0],8))
Dx_r_1 = str(sym.N(Dx_r[1],8))

print(f'Dx_r_1 = {u_r_d};')
print(f'Dx_r_2 = {u_r_q};')

Dx_r_1 = -0.013361535*Dx_d_1 - 0.00027748637*Dx_d_2 + 0.0021866204*Dx_d_3 + 5.2865939e-5*Dx_d_4 + 0.0071983421*Dx_d_5 + 0.00022706741*Dx_d_6 - 2.2443754*Dx_i_1 + 0.68967688*Dx_i_2 - 0.36460717*Dx_r_1 - 0.0049366387*Dx_r_2;
Dx_r_2 = 0.00027748637*Dx_d_1 - 0.013361535*Dx_d_2 - 5.2865939e-5*Dx_d_3 + 0.0021866204*Dx_d_4 - 0.00022706741*Dx_d_5 + 0.0071983421*Dx_d_6 - 0.68967688*Dx_i_1 - 2.2443754*Dx_i_2 + 0.0049366387*Dx_r_1 - 0.36460717*Dx_r_2;


In [356]:
for it in range(6):
    print(f'xD[{it+it_ini}] = {Δx_o_kp1[it]};')

xD[4] = 13.7996619184135*Dx_r_1 + 0.103767816974477*Dx_r_2 + 0.290189894236834*i_sd - 0.290189894236834*i_sd_0 + 0.00455866714091358*i_sq - 0.00455866714091358*i_sq_0 + 0.004003346842446*v_md - 0.004003346842446*v_md_0 + 6.2889597697231e-5*v_mq - 6.2889597697231e-5*v_mq_0 + 0.768729374614947*xD[4] + 0.0120761660206275*xD[5] - 0.0376316155873075*xD[6] - 0.000591164657503082*xD[7] - 0.060612006008949*xD[8] - 0.000952169478073937*xD[9];
xD[5] = -0.103767816974477*Dx_r_1 + 13.7996619184135*Dx_r_2 - 0.00455866714091368*i_sd + 0.00455866714091368*i_sd_0 + 0.290189894236833*i_sq - 0.290189894236833*i_sq_0 - 6.28895976972104e-5*v_md + 6.28895976972104e-5*v_md_0 + 0.00400334684244601*v_mq - 0.00400334684244601*v_mq_0 - 0.0120761660206275*xD[4] + 0.768729374614947*xD[5] + 0.000591164657503061*xD[6] - 0.0376316155873075*xD[7] + 0.000952169478074047*xD[8] - 0.060612006008949*xD[9];
xD[6] = 86.1438235366479*Dx_r_1 + 0.886477352734974*Dx_r_2 - 4.2224902133574*i_sd + 4.2224902133574*i_sd_0 - 0.066332

In [346]:
z_obs_list

['v_md_g01', 'v_mq_g01', 'i_sd_g01', 'i_sq_g01']

In [338]:
K_o

matrix([[ 2.20758392e-02,  3.46794994e-04,  2.30800093e-01,
          3.62569759e-03],
        [-3.46794994e-04,  2.20758392e-02, -3.62569759e-03,
          2.30800093e-01],
        [ 1.30196616e+00,  2.04529188e-02, -1.04919893e+01,
         -1.64821339e-01],
        [-2.04529188e-02,  1.30196616e+00,  1.64821339e-01,
         -1.04919893e+01],
        [ 5.02642205e-02,  7.89613473e-04,  7.69087392e-01,
          1.20817902e-02],
        [-7.89613473e-04,  5.02642205e-02, -1.20817902e-02,
          7.69087392e-01]])

In [357]:
import sympy as sym

x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6 = sym.symbols('xD[4],xD[5],xD[6],xD[7],xD[8],xD[9]')
x_r_1,x_r_2 = sym.symbols('Dx_r_1,Dx_r_2')
x_i_1,x_i_2 = sym.symbols('Dx_i_1,Dx_i_2')

x_e = sym.Matrix([x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6,x_r_1,x_r_2,x_i_1,x_i_2])
u_r = -K_c * x_e

u_r_d = str(sym.N(u_r[0],8))
u_r_q = str(sym.N(u_r[1],8))

print(f'Du_r_1 = {u_r_d};')
print(f'Du_r_2 = {u_r_q};')
print('\nWarning: Control output is eta_dq!!')

Du_r_1 = -2.2443754*Dx_i_1 + 0.68967688*Dx_i_2 - 0.36460717*Dx_r_1 - 0.0049366387*Dx_r_2 - 0.013361535*xD[4] - 0.00027748637*xD[5] + 0.0021866204*xD[6] + 5.2865939e-5*xD[7] + 0.0071983421*xD[8] + 0.00022706741*xD[9];
Du_r_2 = -0.68967688*Dx_i_1 - 2.2443754*Dx_i_2 + 0.0049366387*Dx_r_1 - 0.36460717*Dx_r_2 + 0.00027748637*xD[4] - 0.013361535*xD[5] - 5.2865939e-5*xD[6] + 0.0021866204*xD[7] - 0.00022706741*xD[8] + 0.0071983421*xD[9];



In [341]:
B_o

array([[ 1.37996619e+01,  1.03767817e-01],
       [-1.03767817e-01,  1.37996619e+01],
       [ 8.61438235e+01,  8.86477353e-01],
       [-8.86477353e-01,  8.61438235e+01],
       [ 1.18794716e+00,  1.39161908e-02],
       [-1.39161908e-02,  1.18794716e+00]])